In [11]:
import spacy 
from spacy.matcher import PhraseMatcher
import plac
from pathlib import Path
import random


# PRACTICE

In [13]:

nlp = spacy.load('en')
doc = nlp("Hello everyone, I've some good news to give you")
#_pos 词性
# [x.is_pos for x in doc ]
cleaned = [y for y in doc if not y.is_stop and y.pos_ != 'PUNCT']
raw = [(x.lemma_,x.pos_) for x in cleaned]
print(raw)

to_analyze = ('Hello Code & Supply,' 'my name is Josh and tonight' 'we \'re in Pittsburgh')
doc = nlp(to_analyze)
#Entity Name 
# doc.ents
ents = [(x.text,x.label_) for x in doc.ents]
ents

[('hello', 'INTJ'), ('-PRON-', 'PRON'), ('have', 'VERB'), ('good', 'ADJ'), ('news', 'NOUN')]


[('Josh', 'PERSON'), ('Pittsburgh', 'GPE')]

# PROJECT

In [41]:
def offseter(lbl,doc,matchitem):
        o_one = len(str(doc[0:matchitem[1]]))
        subdoc = doc[matchitem[1]:matchitem[2]]
        o_two = o_one + len(str(subdoc))
        return (o_one,o_two,lbl)
    
nlp = spacy.load('en')
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')
# print(nlp.vocab) 
label = 'CIADIR'
matcher = PhraseMatcher(nlp.vocab)
# add match ID "label" with no callback and one pattern
#matcher.add('HelloWorld', None, pattern)
#第二个参数允许您传递一个可选的回调函数，以调用成功的匹配。
for i in ['Gina Haspel','Gina','Haspel',]:
    matcher.add(label, None, nlp(i))
#如果你知道你想要匹配的内容的上下文，但是对于特定的token和它的字符非常少，那么通配符就显得很有用了。例如，假设您试图从数据中提取用户的用户名。您只知道它们被列为“用户名:{用户名}”。名称本身可能包含任何字符，但没有空格——因此您将知道它将作为一个token处理。
#[{'ORTH': 'User'}, {'ORTH': 'name'}, {'ORTH': ':'}, {}]
test = 'Gina Haspel was nomiated in 2018'
one = nlp(test)
matched = matcher(one)
#返回match_id, start, end
matched

[(17539557946531887154, 0, 1),
 (17539557946531887154, 0, 2),
 (17539557946531887154, 1, 2)]

In [48]:
res = []
to_train_ents = []
matches = matcher(one)
res = [offseter(label,test,x) for x in matches]
to_train_ents.append((test,dict(entities = res)))
to_train_ents

[('Gina Haspel was nomiated in 2018',
  {'entities': [(0, 1, 'CIADIR'), (0, 2, 'CIADIR'), (1, 2, 'CIADIR')]})]

In [50]:
optimizer = nlp.begin_training()
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']


# SIMILAFRITY

In [25]:
# pip install spacy
# python -m spacy download en_core_web_sm

import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = (u"When Sebastian Thrun started working on self-driving cars at "
        u"Google in 2007, few people outside of the company took him "
        u"seriously. “I can tell you very senior CEOs of major American "
        u"car companies would shake my hand and turn away because I wasn’t "
        u"worth talking to,” said Thrun, now the co-founder and CEO of "
        u"online higher education startup Udacity, in an interview with "
        u"Recode earlier this week.")
doc = nlp(text)

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

# Determine semantic similarities
doc1 = nlp(u"my fries were super gross")
doc2 = nlp(u"such disgusting fries")
similarity = doc1.similarity(doc2)
print(doc1.text, doc2.text, similarity)

Sebastian Thrun PERSON
Google ORG
2007 DATE
American NORP
Thrun PERSON
Recode ORG
earlier this week DATE
my fries were super gross such disgusting fries 0.7139701182549687
